In [1]:
import requests
import p03_wiki
import os
import json
import base64
import glob
import gzip
import functools

In [2]:
client = p03_wiki.login()

In [4]:
cat = client.categories["Expansion Pack 1 Cards"]

In [10]:
page = client.pages["Asmanteus"]

In [13]:
[c.name for c in list(page.categories())]

['Category:Pages using Tabber parser tag',
 'Category:Rare Cards',
 'Category:Transformer Cards',
 'Category:Cards',
 'Category:Expansion Pack 1 Cards',
 'Category:Unobtainable Cards']

In [3]:
def get_portrait_b64(card_name):
    with open(os.path.join(os.environ["INSCRYPTION_PATH"], "cardexports", f"card_{card_name}.png"), "rb") as f:
        return base64.b64encode(f.read()).decode('ascii')
    
def get_ability_b64(ability_name):
    with open(os.path.join(os.environ["INSCRYPTION_PATH"], "cardexports", f"ability_{ability_name}.png"), "rb") as f:
        return base64.b64encode(f.read()).decode('ascii')

In [7]:
def get_abilities_request_body(data):
	ability_dict = p03_wiki.get_ability_dictionary()
	retval = [{"referenceId": "", "extraData": "$undefined"}] * 5
	for i, ab in enumerate(data['abilities']):
		ability_data = ability_dict[ab]

		retval[i] = {
			"referenceId": "custom",
			"nameOnCard": "",
			"description": "",
			"gameId": "",
			"imageBase64": "data:image/png;base64," + get_ability_b64(ability_data['name']),
			"gbcImageBase64": "$undefined",
			"pixelProfilgateImageBase64": "$undefined",
			"augmentedImageBase64": "$undefined"
		}

	return retval

def auto_gemify(card_name):
	return 'sentinel' in card_name.lower() or card_name in ['GemRipper', 'TechMoxTriple']

def is_conduit(card_name):
	data = p03_wiki.get_card_data(card_name)
	ability_dict = p03_wiki.get_ability_dictionary()
	for ab in data['abilities']:
		ab_data = ability_dict[ab]
		if ab_data['conduit']:
			return True
	return False

def get_secondary_data(card_name):
	secondary = []
	if auto_gemify(card_name):
		secondary.append("gemified")
	if is_conduit(card_name):
		secondary.append("circuit")
	if p03_wiki.get_card_quality(p03_wiki.get_card_data(card_name)) == "Rare":
		secondary.append("pkcm")
	return secondary

def get_request_body(card_name):
	data = p03_wiki.get_card_data(card_name)
	return ["p03",
		{
			"name": data['displayedName'],
			"rare": p03_wiki.get_card_quality(data) == "Rare",
			"terrain": False,
			"health": data['baseHealth'],
			"useCustomPortrait": True,
			"portrait": {
				"imageBase64": "data:image/png;base64," + get_portrait_b64(card_name)
			},
			"portraitId": "",
			"useStatIcon": False,
			"attack": data['baseAttack'],
			"statIcon": "",
			"temple": "tech",
			"bloodCost": 0,
			"boneCost": 0,
			"energyCost": data['energyCost'],
			"gemsCost": [],
			"abilities": get_abilities_request_body(data),
			"tribes": "$W1",
			"flags": "$W2",
			"evolvesInto": "",
			"evolvesIntoTurns": 1,
			"evolvesIntoName": "",
			"iceCubedInto": "",
			"description": ""
		},
		{
			"locale": "default",
			"border": False,
			"scanlines": True
		}
	]

In [5]:
boundary = "---------------------------4172021632017129141242411954"
def generate_request_text(request, boundary, secondary_data=[]):
    second_string = ""
    if len(secondary_data) > 0:
        second_string = ",".join(f'"{s}"' for s in secondary_data)
    return (
        f"--{boundary}\r\n"
        'Content-Disposition: form-data; name="1"\r\n\r\n'
        '[]\r\n'
        f"--{boundary}\r\n"
        'Content-Disposition: form-data; name="2"\r\n\r\n'
        f'[{second_string}]\r\n'
        f"--{boundary}\r\n"
        'Content-Disposition: form-data; name="0"\r\n\r\n'
        f'{json.dumps(request, separators=(",", ":"))}\r\n'
        f"--{boundary}--\r\n"
    )

In [8]:
for fname in glob.glob(os.path.join(os.environ["INSCRYPTION_PATH"], 'cardexports', "card*.json")):

    card_name = os.path.basename(fname).replace('card_', '').replace('.json', '')
    if "!" in card_name:
        continue

    data = p03_wiki.get_card_data(card_name)
    if p03_wiki._special_enums()['CardTemple'][str(data['temple'])] != 'Tech':
        continue

    try:
        second_data = get_secondary_data(card_name)
        # Temporary for this rebuild:
        if "pkcm" not in second_data:
            continue
        request_body = get_request_body(card_name)
        text = generate_request_text(request_body, boundary, secondary_data=second_data)
        print(f'Requesting {card_name} from generator.cards')
        post_req = requests.post(
            "https://generator.cards/",
            verify=True,
            headers={
                # "Accept": "text/x-component",
                # "Accept-Encoding": "gzip, deflate, br",
                "Content-Length": str(len(text)),
                "Next-Action": "ee0c6925fd7cec9fab62399a6430fb0316fac4a2",
                "Next-Router-State-Tree": "%5B%22%22%2C%7B%22children%22%3A%5B%22(front)%22%2C%7B%22children%22%3A%5B%22__PAGE__%22%2C%7B%7D%5D%7D%5D%7D%2Cnull%2Cnull%2Ctrue%5D",
                "Next-Url": "/",
                #"Connection": "keep-alive",
                "Content-Type": f"multipart/form-data; boundary={boundary}",
            },
            data=text
        )
        post_req.raise_for_status()
        b64img = post_req.content.decode().split("base64,")[1].split(":")[0]
        try:
            wraparound = post_req.content.decode().split(",data:image/png")[0].split("2:")[1]
        except Exception:
            wraparound = ""
        with open(os.path.join('cardexports', f"{card_name}.png"), "wb") as f:
            try:
                f.write(base64.b64decode(b64img))
            except Exception:
                f.write(base64.b64decode(b64img+wraparound))
    except Exception as ex:
        print(f'Failed to generate {card_name}: {ex}')

Requesting AboveCurve from generator.cards
Requesting BombMaiden from generator.cards
Requesting EnergyConduit from generator.cards
Requesting EnergyRoller from generator.cards
Failed to generate Librarian: 59
Requesting Ouroboros_Part3 from generator.cards
Requesting P03KCMXP1_BeastMaster from generator.cards
Requesting P03KCMXP1_BuckingBull from generator.cards
Requesting P03KCMXP1_CopyPasta from generator.cards
Requesting P03KCMXP1_EmeraldTitan from generator.cards
Requesting P03KCMXP1_GoogleBot from generator.cards
Requesting P03KCMXP1_MantisBot from generator.cards
Requesting P03KCMXP1_Necrobot from generator.cards
Requesting P03KCMXP1_RoboAngel from generator.cards
Requesting P03KCMXP1_RubberDuck from generator.cards
Requesting P03KCMXP1_RubyTitan from generator.cards
Requesting P03KCMXP1_SapphireTitan from generator.cards
Requesting P03KCMXP1_Worm from generator.cards
Requesting P03KCMXP1_ZombieProcess from generator.cards
Requesting P03KCMXP2_DrZambot from generator.cards
Reque

In [18]:
from PIL import Image
import numpy as np

In [14]:
im.getbbox()

(439,1080-149,973,692)

(71, 151, 1608, 917)

In [17]:
for fname in glob.glob(os.path.join(os.environ["INSCRYPTION_PATH"], 'cardexports', "p03face*.png")):
    im = Image.open(fname)
    im2 = im.crop((439,149,439+973,149+692))
    im2.save(os.path.join('cardexports', os.path.basename(fname)))

In [52]:
#@functools.cache
def _get_eyes(fname='C:\\GIT\\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension\\ascensionicon_activated_default.png'):
    eyes = Image.open(fname)
    eyes = eyes.convert('RGBA')

    eyes = np.array(eyes)
    r, g, b, a = eyes.T
    eyes = a > 0
    print(fname)
    return eyes

def get_eyes(fname):
    for suffix in ['_activated', '_active', 'activated', 'active']:
        try:
            return _get_eyes(fname.replace('.png', f'{suffix}.png'))
        except:
            continue
    try:
        return _get_eyes(fname.replace('ascensionicon', 'ascensionicon_activated'))
    except:
        pass
    return _get_eyes()

In [53]:
dirs = [
    os.path.abspath(os.path.join('..', 'P03KayceeRun', 'assets')),
    'C:\\GIT\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension'
]
for dir in dirs:
    for fname in glob.glob(os.path.abspath(os.path.join(dir, 'ascensionicon*.png'))):
        if 'activate' in fname:
            continue

        print(fname)

        im = Image.open(fname)
        im = im.convert('RGBA')

        data = np.array(im)
        r, g, b, a = data.T

        new_r = a > 0
        data[..., :][new_r.T] = (155, 37, 49, 255)

        eyes = get_eyes(fname)
        data[..., :][eyes.T] = (247, 248, 206, 255)

        im2 = Image.fromarray(data)
        im2.save(os.path.join('cardexports', os.path.basename(fname)))

c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bomb.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bombactivated.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_bounthunter.png
C:\GIT\Inscryption-Kaycee\globalgamemanagers\Assets\Resources\art\ui\ascension\ascensionicon_activated_default.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_broken_bridge.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_broken_bridge_activated.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle_active.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_conveyorbattle_active.png
C:\GIT\Inscryption-Kaycee\globalgamemanagers\Assets\Resources\art\ui\ascension\ascensionicon_activated_default.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon_eccentricpainter.png
c:\GIT\P03KayceeMod-Main\P03KayceeRun\assets\ascensionicon

<>:3: DeprecationWarning: invalid escape sequence '\I'
<>:3: DeprecationWarning: invalid escape sequence '\I'
C:\Users\divis\AppData\Local\Temp\ipykernel_6564\1142402360.py:3: DeprecationWarning: invalid escape sequence '\I'
  'C:\\GIT\Inscryption-Kaycee\\globalgamemanagers\\Assets\\Resources\\art\\ui\\ascension'


In [48]:
site = p03_wiki.login()

In [51]:
for fname in glob.glob(os.path.abspath(os.path.join('cardexports', 'ascensionicon*.png'))):
    try:
        site.Pages[f"File:{os.path.basename(fname)}"].delete()
    except:
        continue

In [9]:
outstr = ""
cur_ach = {}
for idx, line in enumerate(teststr.split("\n")):
    if idx % 9 == 2:
        cur_ach['title'] = line.split('"')[1]
    if idx % 9 == 3:
        cur_ach['desc'] = line.split('"')[1]
    if idx % 9 == 6:
        cur_ach['file'] = line.split('"')[1]
    if idx % 9 == 8:
        outstr += "|-\n"
        outstr += f"| [[File:{cur_ach['file']}|75px]]\n"
        outstr += "| [[File:sticker_null.png|75px]]\n"
        outstr += f"| {cur_ach['title']}\n"
        outstr += f"| {cur_ach['desc']}\n"
        cur_ach.clear()
print(outstr)

|-
| [[File:achievement_first_win.png|75px]]
| [[File:sticker_null.png|75px]]
| Uber Apocalypse
| Beat 
|-
| [[File:achievement_skull.png|75px]]
| [[File:sticker_null.png|75px]]
| Scarlet Skull
| Win a run with every challenge skull from the first page enabled
|-
| [[File:achievement_nft.png|75px]]
| [[File:sticker_null.png|75px]]
| Avatar
| Take ownership of one of G0lly!'s beloved trinkets
|-
| [[File:achievement_six.png|75px]]
| [[File:sticker_null.png|75px]]
| Book of the Dead
| During The Archivist's boss fight, survive 6 turns during phase 2
|-
| [[File:achievement_camera.png|75px]]
| [[File:sticker_null.png|75px]]
| Feral Spirit
| Defeat the Photographer without using the camera
|-
| [[File:achievement_loop.png|75px]]
| [[File:sticker_null.png|75px]]
| Gallery Showcase
| Create an infinite loop in the Unfinished Boss fight
|-
| [[File:achievement_bounty.png|75px]]
| [[File:sticker_null.png|75px]]
| Battle Frenzy
| Destroy 30 Bounty Hunters in your lifetime
|-
| [[File:achievemen

In [8]:
teststr = """FIRST_WIN = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Uber Apocalypse",
                "Beat \"P03 in Kaycee's Mod\" for the first time",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_first_win.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [🚫] sticker

            SKULLSTORM = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Scarlet Skull",
                "Win a run with every challenge skull from the first page enabled",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_skull.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [red skull] sticker

            CONTROL_NFT = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Avatar",
                "Take ownership of one of G0lly!'s beloved trinkets",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_nft.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [an ages dead cat meme, like 2010] sticker

            SURVIVE_SIX_ARCHIVIST = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Book of the Dead",
                "During The Archivist's boss fight, survive 6 turns during phase 2",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_six.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [101010 strip] sticker

            DONT_USE_CAMERA = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Feral Spirit",
                "Defeat the Photographer without using the camera",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_camera.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [broken camera] sticker

            CANVAS_ENOUGH = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Gallery Showcase",
                "Create an infinite loop in the Unfinished Boss fight",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_loop.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [P03's annoyed face as an a rageface] sticker

            KILL_30_BOUNTY_HUNTERS = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Battle Frenzy",
                "Destroy 30 Bounty Hunters in your lifetime",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_bounty.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [cowboy hat] sticker

            ALL_QUESTS_COMPLETED = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Calling for Backup",
                "Complete all NPC quests that appear in a single run",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_quests.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [smiley face with a wink and thumbs up] sticker

            KILL_QUEST_CARD = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Accelerated Fuse",
                "Skeleclock and destroy a card given to you by an NPC",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_kill_quest_card.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [scorched companion cube] sticker

            TURBO_RAMP = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Mana Blink",
                "Have six energy available on turn two",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_six_energy.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [leaky battery] sticker

            MASSIVE_OVERKILL = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Aimless Assault",
                "Earn 15 Robobucks in a single battle",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_money.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [tophat and monocle] sticker

            FULLY_UPGRADED = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Rebuild From Scrap",
                "Have a card in your deck that is gemified, skeleclocked, transformable, and can complete a circuit",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_plus_four.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [💪] sticker

            MAX_SP_CARD = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Buy Your Buddy",
                "Give a card with the maximum amount of SP to the Bot Builder",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_bab.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [Sir Fire, Esquire] sticker

            SIX_SHOOTER = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Me Smash",
                "Strike P03 six separate times in a single turn",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_six_shooter.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [smoking revolver] sticker

            SCALES_TILTED_3X = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Absolution",
                "Have the scales tiled 4 points against you on three different turns",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_scales.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [that one "hang in there!" cat poster] sticker

            AVOID_BOUNTY_HUNTERS = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "The Perfect Crime",
                "Win five battles with bounty hunters without killing them",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_crime.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [peace sign] sticker

            PLASMA_JIMMY_CRAZY = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Blastenheimer 5000 Ultra Cannon",
                "Deal 6 damage with Plasma Jimmy in a single turn",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_plasma.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [orange wizard hat] sticker

            FULLY_OVERCLOCKED = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Double Time",
                "Skeleclock every card in your deck",
                false,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_brittle.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [guillotine] sticker

            MYCOLOGISTS_COMPLETED = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "One And The Same",
                "Complete the Mycologist's experiment and create an abomination of science",
                true,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_mushroom.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [trippy rainbow shrooms] sticker

            FAST_GENERATOR = ModdedAchievementManager.New(
                P03Plugin.PluginGuid,
                "Burning Adrenaline",
                "Repair the generator in three turns or less",
                true,
                grp.ID,
                TextureHelper.GetImageAsTexture("achievement_generator.png", typeof(P03AchievementManagement).Assembly)
            ).ID; // [hermes boots, shoe with wings on it] sticker
            
            """